In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 39.3 MB/s 
     |████████████████████████████████| 163 kB 55.4 MB/s 


In [4]:
from transformers import pipeline

In [5]:
nlp = pipeline("question-answering", model="deepset/roberta-base-squad2", tokenizer="deepset/roberta-base-squad2")

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [8]:
def get_answer(question: str, context: str):
    config = {
        "question": question,
        "context": context
    }
    result = nlp(config)
    return result

In [17]:
import pandas as pd
dataset = pd.read_csv("/content/FAQs.csv")
test = pd.read_csv("/content/FAQs_test.csv")
dataset.head()

,Question,Answer
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.
1,Where was he born?,"He was born in Ulm, Germany."
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey..."
3,Who were his parents?,His father was Hermann Einstein and his mother...
4,Did he have any sisters and brothers?,He had one sister named Maja.


In [16]:
raw_answers = []
for i, j in zip(dataset.Question, dataset.Answer):
    # print(i, j)
    raw_answers.append(get_answer(i, j))

raw_answers

[{'score': 0.9651150703430176,
  'start': 28,
  'end': 41,
  'answer': '14 March 1879'},
 {'score': 0.9313827753067017,
  'start': 15,
  'end': 27,
  'answer': 'Ulm, Germany'},
 {'score': 0.978484034538269,
  'start': 8,
  'end': 21,
  'answer': '18 April 1955'},
 {'score': 0.5722765326499939,
  'start': 15,
  'end': 31,
  'answer': 'Hermann Einstein'},
 {'score': 0.03699028864502907,
  'start': 0,
  'end': 28,
  'answer': 'He had one sister named Maja'},
 {'score': 0.14178985357284546,
  'start': 148,
  'end': 173,
  'answer': 'He married Elsa Löwenthal'},
 {'score': 0.5658432841300964,
  'start': 87,
  'end': 102,
  'answer': 'Munich, Germany'},
 {'score': 0.6494642496109009, 'start': 328, 'end': 332, 'answer': '1922'},
 {'score': 0.36804699897766113,
  'start': 80,
  'end': 105,
  'answer': 'Albert Einstein could not'},
 {'score': 0.1970616728067398,
  'start': 96,
  'end': 144,
  'answer': 'discovery of the law of the photoelectric effect'}]

In [18]:
paragraph = ""
for i in dataset.Answer:
    paragraph += i
paragraph

'Albert Einstein was born on 14 March 1879.He was born in Ulm, Germany. He died 18 April 1955 in Princeton, New Jersey, USA.His father was Hermann Einstein and his mother was Pauline Einstein (born Koch).He had one sister named Maja.He was married to Mileva Marić between 1903 and 1919. They had three children, Lieserl (born 1902), Hans Albert (born 1904) and Eduard (born 1910). He married Elsa Löwenthal in 1919 and they lived together until her death in 1936.He received his main education at the following schools: Catholic elementary school in Munich, Germany (1885-1888)Luitpold Gymnasium in Munich, Germany (1888-1894) Cantonal school in Aarau, Switzerland (1895-1896) Swiss Federal Institute of Technology in Zurich, Switzerland (1896-1900) Ph.D. from Zurich University, Switzerland (1905)The Nobel Prize Awarding Institution, the Royal Swedish Academy of Sciences, decided to reserve the Nobel Prize in Physics in 1921, and therefore no Physics Prize was awarded that year. According to the

In [21]:
final_answer_json = []
for i in test.Question:
    final_answer_json.append(get_answer(i, paragraph))
final_answer_json

[{'score': 0.9660661220550537,
  'start': 79,
  'end': 92,
  'answer': '18 April 1955'},
 {'score': 0.1890774667263031,
  'start': 203,
  'end': 231,
  'answer': 'He had one sister named Maja'},
 {'score': 0.2545149326324463,
  'start': 391,
  'end': 405,
  'answer': 'Elsa Löwenthal'},
 {'score': 0.8945491909980774,
  'start': 138,
  'end': 154,
  'answer': 'Hermann Einstein'},
 {'score': 0.0519428588449955,
  'start': 676,
  'end': 713,
  'answer': 'Swiss Federal Institute of Technology'}]

In [23]:
Answer = []
for i in range (len(final_answer_json)):
    Answer.append(final_answer_json[i]['answer'])
Answer

['18 April 1955',
 'He had one sister named Maja',
 'Elsa Löwenthal',
 'Hermann Einstein',
 'Swiss Federal Institute of Technology']

In [24]:
test['Answer'] = Answer
test.head()

,Question,Answer
0,What is the date of his death?,18 April 1955
1,Did Einstein have siblings?,He had one sister named Maja
2,Who was his wife?,Elsa Löwenthal
3,What was Einstein's father's name?,Hermann Einstein
4,At what institutions did he study?,Swiss Federal Institute of Technology


In [26]:
test.to_csv('submission.csv', index=False)